[Go to Home Page](https://weaviate.oneblink.ai)


# Audio Genre Classifier
## Introduction
Welcome to the Audio Genre Classifier workshop! In this lab, we'll use Weaviate's img2vec module to build a classifier that converts images of audio spectrograms into vectors using neural networks. We'll develop the frontend using HTML, CSS, and JavaScript and the backend with Flask. You'll learn to set up your environment, create data schemas, import data, and implement a Flask application for audio file upload. Let's get started!
Explore the project's origin [here](https://github.com/weaviate/weaviate-examples/tree/main/audio-genre-classification).

## Environment Setup
First, let's set up our working environment by importing necessary libraries and initializing our Weaviate client.

In [ ]:
import weaviate
import os
client = weaviate.Client("http://localhost:8085")
print("Weaviate Client Connected")


## Schema Creation
Now, we'll create a schema for our audio genres. This step involves defining the data structure in Weaviate.

In [ ]:
#Checking if Audiogenres schema already exists, then delete it
current_schemas = client.schema.get()['classes']
for schema in current_schemas:
    if schema['class']=='Audiogenres':
        client.schema.delete_class('Audiogenres')
        print("Deleted existing schema: Audiogenres")


class_obj = {
        "class": "Audiogenres",
        "description": "Each object contains an image of a spectrogram and a label about genre of that spectrogram",
        "moduleConfig": {
            "img2vec-neural": {
                "imageFields": [
                    "image"
                ]
            }
        },
        "properties": [
            {
                "dataType": [
                    "blob"
                ],
                "description": "spectrogram image",
                "name": "image"
            },
            {
                "dataType": [
                    "string"
                ],
                "description": "label name (genre) of the given spectrogram.",
                "name": "labelName"
            }

        ],

        "vectorIndexType": "hnsw",
        "vectorizer": "img2vec-neural"
    }

client.schema.create_class(class_obj)
print("Created new schema class: Audiogenres")



## Data Importing
Next, we will import the spectrogram images. We'll configure the batch processing for efficiency and then load images from each genre folder.

In [ ]:
#Configure batch process - for faster imports 
#see: https://weaviate.io/developers/weaviate/current/restful-api-references/batch.html
client.batch.configure(
  batch_size=10, 
  # dynamically update the `batch_size` based on import speed
  dynamic=True,
  timeout_retries=3,
)

folders = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']; 
for fol in folders:
    for img in os.listdir(f"images_original/{fol}"):
        print(f"Importing {img}")
        
        encoded_image = weaviate.util.image_encoder_b64(f"images_original/{fol}/{img}")
        
        data_properties = {
            "labelName": fol,
            "image": encoded_image
        }
        
        try:
            client.data_object.create(data_properties, "Audiogenres")
        except BaseException as error:
            print("Import Failed at: ",i)
            print("An exception occurred: {}".format(error))
            # Stop the import on error
            break

    print(f"All images from {fol} added.")

client.batch.flush()

## Flask Application Integration
Finally, let's integrate the Flask application for audio file uploads. This application will allow users to upload audio files, which will then be processed to generate spectrograms and classify the audio genre.

In [ ]:
!python3.11 upload.py

## Conclusion
Congratulations on completing the lab session! You've successfully built an Audio Genre Classifier using Weaviate and integrated a Flask application for audio file uploads. For more information and resources, refer to the [GitHub repository](https://github.com/weaviate/weaviate-examples/tree/main/audio-genre-classification). Happy coding!

[Go to Home Page](https://weaviate.oneblink.ai)
